In [1]:
# !pip uninstall nvidia_cublas_cu11 --yes
# !pip install langchain
# !pip install "google-cloud-aiplatform>1.25" "shapely<2.0.0"
# !pip install scann
# !pip install cuda-python
# !pip install tensorrt 
# !pip install tensorflow
# !pip install sentence_transformers

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if
            x.device_type == 'GPU']

print(get_available_gpus())

2023-11-30 18:47:15.912768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 18:47:17.609931: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-11-30 18:47:17.610312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

[]


2023-11-30 18:47:19.128969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 18:47:19.129637: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-11-30 18:47:19.129660: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-30 18:47:19.129682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gen-ai-notebook): /proc/driver/nv

In [3]:
import pandas as pd
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
import json
import time
import scann

2023-12-05 15:41:51.581676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 15:41:52.606197: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-05 15:41:52.606349: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [4]:
# Import Vertex AI SDK
PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
LOCATION = "europe-west2" 

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
df = pd.read_csv('text_data.csv')
df

,file_name,text,url
0,find-iban-bic.txt,Find your IBAN and BICIBAN is your Internation...,https://www.lloydsbank.com/business/help-and-s...
1,cbo-system-compatibility.txt,Software you should use with Commercial Bankin...,https://www.lloydsbank.com/business/help-and-s...
2,transaction-not-recognised.txt,Find out more about a transactionIf you don’t ...,https://www.lloydsbank.com/business/help-and-s...
3,ob-card-reader-not-working.txt,My Online for Business card reader isn’t worki...,https://www.lloydsbank.com/business/help-and-s...
4,cbo-account-suspended.txt,My Commercial Banking Online access has been s...,https://www.lloydsbank.com/business/help-and-s...
...,...,...,...
60,order-card-reader-for-ob.txt,Order a card reader for Online for BusinessIn ...,https://www.lloydsbank.com/business/help-and-s...
61,make-a-payment.txt,Make a paymentPay a person or business in the ...,https://www.lloydsbank.com/business/help-and-s...
62,log-on-to-o4b.txt,Log on to Online for BusinessLog on securely u...,https://www.lloydsbank.com/business/help-and-s...
63,confirmation-of-payee.txt,NaN,https://www.lloydsbank.com/business/help-and-s...


In [8]:
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

def get_embedding(text):
    get_embedding.counter += 1
    try:
        if get_embedding.counter % 100 == 0:
            time.sleep(3)
        return model.get_embeddings([text])[0].values #Send request to embedding model
    except:
        return []


get_embedding.counter = 0

# This may take several minutes to complete.
df["embedding"] = df["text"].apply(lambda x: get_embedding(x))
df.to_csv('text_data_g_embedding.csv', index=False)

In [9]:
# this has stopped working after some rows are empty
# Import and apply embeddings from HuggingFace
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings()

# df['embedding']=df.apply(lambda row: embeddings.embed_query(row['text']),axis=1)
# df.to_csv('text_data_h_embedding.csv', index=False)

AttributeError: 'float' object has no attribute 'replace'